# Підручник: Основні засоби приватного глибинного навчання

Ласкаво просимо до вступного підручника PySyft щодо збереження конфіденційності, децентралізованого глибинного навчання. Ця серія зошитів - покрокове керівництво для ознайомлення з новими інструментами та прийомами, необхідними для глибинного навчання на секретних / приватних даних / моделей, без централізації їх за одним повноваженням.

**Сфера застосування:** Зауважте, що ми не прото будемо говорити про те, як децентралізувати / шифрувати дані, але ми розглянемо, як PySyft може бути використаний для децентралізації всієї екосистеми навколо даних,  навіть включаючи бази даних, де дані зберігаються та робляться запити, та нейронних мереж, які використовуються для отримання інформації з даних. У міру створення нових розширень до PySyft ці ноутбуки будуть розширені новими навчальними посібниками, щоб пояснити нову функціональність.

Автори:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

Переклад:
- Bogdan Ivanyuk-Skulskiy - Facebook: [Bogdan Ivanyuk](https://www.facebook.com/bogdan.ivanyuk) - LinkedIn: [Bogdan Ivanyuk](https://www.linkedin.com/in/bogdan-ivanyuk-skulskiy-982739163/)

## Зміст:

- Частина 1: Основні засоби приватного глибокого навчання


## Навіщо дивитися ці зошити?

**1) Кар'єрні переваги** - За останні 20 років цифрова революція зробила дані все більш доступними у більш великих кількостях, оскільки аналогічні процеси стають оцифрованими. Однак, згідно з новим регулюванням, таким як [GDPR](https://eugdpr.org/), підприємства опиняються під тиском, щоб мати менше свободи в тому, як вони використовують - а ще важливіше, як вони аналізують - особисту інформацію. ** Підсумок: ** Вчені даних не матимуть доступу до такої кількості даних за допомогою інструментів "старої школи", але, вивчаючи інструменти приватного глибинного навчання, Ви можете випереджати цю криву та мати конкурентну перевагу в Ваша кар'єра. 

**2) Підприємницькі можливості** - У суспільстві існує ціла низка проблем, які глибоке навчання може вирішити, але багато найважливіших не були досліджені, оскільки це вимагало б доступу до неймовірно чутливої інформації про людей. Таким чином, навчання приватного глибокого навчання відкриває для вас безліч нових можливостей запуску, які раніше не були доступні іншим без цих наборів інструментів.

**3) Соціальне благо** - глибинне навчання може використовуватися для вирішення найрізноманітніших проблем у реальному світі, але глибоке навчання на *особистій інформації* - це глибинне навчання про людей, *для людей*. Навчитися робити глибинне вивчення даних, якими ви не володієте, представляє собою не лише кар'єру чи підприємницьку можливість, це можливість допомогти вирішити деякі найбільш особисті і найважливіші проблеми в житті людей - і зробити це в масштабах світу.

## Що я можу зробити?

- Поставити зірочку PySyft на GitHub! - [https://github.com/OpenMined/PySyft](https://github.com/OpenMined/PySyft)
- Зробити відео про матеріал цього зошиту!

# Частина -1: Передумови

- Знати PyTorch - якщо ні, пройдіть курс на  http://fast.ai та повертайтесь
- Прочитайте PySyft Framework статтю https://arxiv.org/pdf/1811.04017.pdf! Це дасть вам підґрунтя щодо побудови PySyft, що допоможе речам мати більше сенсу..

# Частина 0: Налаштування

Для початку вам потрібно буде переконатися, що ви встановили правильні пакети. Для цього перейдіть до readme PySyft і дотримуйтесь інструкцій із налаштування.

- Встановіть Python 3.6 або вище
- Встановіть PyTorch 1.4
- pip install syft[udacity]

Якщо якась частина цього не працює для вас (або будь-який тест не працює) - спочатку перевірте [README](https://github.com/OpenMined/PySyft.git) на щодо встановлення, а потім відкрийте Issue GitHub, або пінг каналу #beginner в нашому slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
# Run this cell to see if things work
import sys

import torch
from torch.nn import Parameter
import torch.nn as nn
import torch.nn.functional as F

import syft as sy
hook = sy.TorchHook(torch)

torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

Якщо ця клітина виконалася, то все йде добре! Давай зробимо це!

# Частина 1: Основні засоби приватної, децентралізованої науки про дані

Отже - перше питання, яке може вас цікавити, - як у світі ми тренуємо модель на даних, до яких ми не маємо доступу?

Що ж, відповідь напрочуд проста. Якщо ви звикли працювати в PyTorch, то ви звикли працювати з torch.Tensor об'єктами!

In [2]:
x = torch.tensor([1,2,3,4,5])
y = x + x
print(y)

tensor([ 2,  4,  6,  8, 10])


Очевидно, що використання цих надзвичайних (і потужних!) тензорів є важливим, але також вимагає, щоб ви мали дані на вашому локальному машині. Тут починається наша подорож.

# Розділ 1.1 - Відправлення тензорів до машини Боба

Зазвичай ми б виконували глибинне навчання на машині, яка зберігає дані. Тепер ми хочемо виконати подібні обчислення на деяких **інших** машинах. Більш конкретно, ми не можемо просто припустити, що дані є на нашій локальній машині.

Таким чином, замість використання Torch тензорів, ми будемо працювати з **покажчиками** на тензори. Дозвольте показати вам, що я маю на увазі. По-перше, давайте створимо машину "прикидатися", що належить людині, яка "прикидається" - ми будемо називати його Боб.

In [3]:
bob = sy.VirtualWorker(hook, id="bob")

Скажімо, машина Боба знаходиться на іншій планеті - можливо, на Марсі! Але, на даний момент машина порожня. Давайте створимо деякі дані, щоб ми могли надіслати їх Бобу та дізнатися про покажчики!

In [4]:
x = torch.tensor([1,2,3,4,5])
y = torch.tensor([1,1,1,1,1])

А тепер - давайте надішлемо наші тензори до Боба !!

In [5]:
x_ptr = x.send(bob)
y_ptr = y.send(bob)

In [6]:
x_ptr

(Wrapper)>[PointerTensor | me:79612369900 -> bob:70145720491]

БУМ! Зараз у Боба два тензори! Не вірите мені? Подивіться самі!

In [7]:
bob._objects

{70145720491: tensor([1, 2, 3, 4, 5]), 85258309307: tensor([1, 1, 1, 1, 1])}

In [8]:
z = x_ptr + x_ptr

In [9]:
z

(Wrapper)>[PointerTensor | me:86902737187 -> bob:43444110328]

In [10]:
bob._objects

{70145720491: tensor([1, 2, 3, 4, 5]),
 85258309307: tensor([1, 1, 1, 1, 1]),
 43444110328: tensor([ 2,  4,  6,  8, 10])}

Коли ми викликали `x.send (bob)`, він повернув новий об'єкт, який ми назвали `x_ptr`. Це наш перший *покажчик* на тензор. Покажчики на тензори НЕ зберігають дані. Натомість вони просто містять метадані про тензора (з даними), що зберігаються на іншій машині. Мета цих тензорів полягає в тому, щоб дати нам інтуїтивно зрозумілий API, щоб повідомити іншій машині для обчислення функцій за допомогою цього тензора. Давайте розглянемо метадані, які містять покажчики.

In [11]:
x_ptr

(Wrapper)>[PointerTensor | me:79612369900 -> bob:70145720491]

Подивіться на метадані!

Є два основні атрибути, характерні для вказівникаs:

- `x_ptr.location : bob`, місцеположення, посилання на місце, на яке вказує вказівник
- `x_ptr.id_at_location : <random integer>`, ідентифікатор, де зберігається тензор у місці розташування

Вони друкуються у форматі `<id_at_location>@<location>`

Є й інші більш загальні ознаки:
- `x_ptr.id : <random integer>`, ідентифікатор нашого тензора вказівника, він був розподілений випадковим чином
- `x_ptr.owner : "me"`, воркер, який володіє тензором вказівникои, ось це місцевий воркер, названий "me"


In [12]:
x_ptr.location

<VirtualWorker id:bob #objects:3>

In [13]:
bob

<VirtualWorker id:bob #objects:3>

In [14]:
bob == x_ptr.location

True

In [15]:
x_ptr.id_at_location

70145720491

In [16]:
x_ptr.owner

<VirtualWorker id:me #objects:0>

Вам може бути цікаво, чому місцевий воркер, який володіє вказівником, також є VirtualWorker, хоча ми його не створили.
Цікаво, що так само, як у нас був об’єкт VirtualWorker для Боба, у нас (за замовчуванням) завжди є і для нас. Цей воркер автоматично створюється, коли ми називаємо `hook = sy.TorchHook()`, і тому вам зазвичай не потрібно створювати його самостійно.

In [17]:
me = sy.local_worker
me

<VirtualWorker id:me #objects:0>

In [18]:
me == x_ptr.owner

True

І нарешті, так само, як ми можемо викликати .send () на тензорі, ми можемо викликати .get () на вказівник до тензора, щоб повернути його !!!

In [19]:
x_ptr

(Wrapper)>[PointerTensor | me:79612369900 -> bob:70145720491]

In [20]:
x_ptr.get()

tensor([1, 2, 3, 4, 5])

In [21]:
y_ptr

(Wrapper)>[PointerTensor | me:25117680280 -> bob:85258309307]

In [22]:
y_ptr.get()

tensor([1, 1, 1, 1, 1])

In [23]:
z.get()

tensor([ 2,  4,  6,  8, 10])

In [24]:
bob._objects

{}

І як бачите ... У Боба більше немає тензорів !!! Вони повернулися до нашої машини!

# Розділ 1.2 - Використання тензорних покажчиків

Отже, надсилання та отримання тензорів від Боба це чудово, але це навряд чи глибинне навчання! Ми хочемо мати можливість виконувати tensor _operations_ на віддалених тензорах. На щастя, тензорні вказівники роблять це досить просто! Ви можете просто використовувати вказівники, як звичайні тензори!

In [25]:
x = torch.tensor([1,2,3,4,5]).send(bob)
y = torch.tensor([1,1,1,1,1]).send(bob)

In [26]:
z = x + y

In [27]:
z

(Wrapper)>[PointerTensor | me:92728388024 -> bob:59511679930]

І вуаля! 

За лаштунками сталося щось дуже потужне. Замість того, щоб x і y обчислювали додавання локально, команда була серіалізована і надіслана Бобу, який виконував обчислення, створив тензор z, а потім повернув покажчик на z назад до нас!

Якщо ми вкажемо .get () на покажчик, ми отримаємо результат назад на нашу машину!

In [28]:
z.get()

tensor([2, 3, 4, 5, 6])

### Torch функції

Цей API поширився на всі операції Torch !!!

In [29]:
x

(Wrapper)>[PointerTensor | me:57089432174 -> bob:64213444148]

In [30]:
y

(Wrapper)>[PointerTensor | me:65506226551 -> bob:74345681526]

In [31]:
z = torch.add(x,y)
z

(Wrapper)>[PointerTensor | me:60677219054 -> bob:282673706]

In [32]:
z.get()

tensor([2, 3, 4, 5, 6])

### Змінні (включаючи зворотне розповсюдження!)

In [33]:
x = torch.tensor([1,2,3,4,5.], requires_grad=True).send(bob)
y = torch.tensor([1,1,1,1,1.], requires_grad=True).send(bob)

In [34]:
z = (x + y).sum()

In [35]:
z.backward()

(Wrapper)>[PointerTensor | me:74324845286 -> bob:19144234957]

In [36]:
x = x.get()

In [37]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [38]:
x.grad

tensor([1., 1., 1., 1., 1.])

Отже, як ви бачите, API дійсно досить гнучкий і здатний виконувати практично будь-які операції, які ви зазвичай виконували в Torch на *віддалених даних*. Це закладає основу для наших більш просунутих протоколів збереження конфіденційності, таких як об'єднане навчання, безпечне обчислення між партіями та диференціальна конфіденційність!

# Вітаємо !!! - Час приєднатися до спільноти!

Вітаємо вас із проходженням цього зошита! Якщо вам сподобалося і ви хотіли б приєднатися до руху до збереження конфіденційності, децентралізованої власності штучного інтелект, ви можете зробити це наступними чином!

### Поставте зірочку PySyft на GitHub

Найпростіший спосіб допомогти нашій спільноті - це поставити зірочку репозиторію на GitHub! Це допомагає підвищити обізнаність про класні інструменти, які ми створюємо.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Приєднуйтесь на Slack!

Найкращий спосіб бути в курсі останніх зрушень - приєднатися до нашої спільноти! Це можна зробити, заповнивши форму на веб-сайті [http://slack.openmined.org](http://slack.openmined.org)

### Приєднуйтесь до проекту!

Найкращий спосіб зробити свій внесок у нашу спільноту - стати учасником коду! У будь-який час ви можете перейти на сторінку Issues PySyft GitHub і відфільтрувати "Projects". Це покаже вам усі квитки вищого рівня, що дають огляд того, до яких проектів ви можете приєднатися! Якщо ви не хочете приєднуватися до проекту, але ви хочете трохи покодити, ви також можете шукати більше "одноразових" міні-проектів, шукаючи проблеми GitHub з позначкою "good first issue".

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Донати

Якщо у вас немає часу зробити свій внесок у нашу кодову базу, але ви все ще хочете надати підтримку, ви також можете стати Backer у нашому Open Collective. Усі пожертвування спрямовуються на наш веб-хостинг та інші витрати спільноти, такі як хакатони та зустрічі!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)